In [1]:
with open("defects.csv") as f:
    lines = f.readlines()
    defects = [line.strip().split(",") for line in lines[1:]]

# Defects never occur at precise integer coordinates, so a defect will never be shared by two tiles.
defect_counts = {}
for x, y in defects:
    x = int(float(x))
    if x not in defect_counts:
        defect_counts[x] = {"a": 0, "b": 0, "c": 0}

    defect_counts[x][y] += 1

print(len(defect_counts))

322


In [3]:
dough_length = 500

biscuits = {
    0: {"length": 4, "value": 6, "defects": {"a": 4, "b": 2, "c": 3}, "id": 0},
    1: {"length": 8, "value": 12, "defects": {"a": 5, "b": 4, "c": 4}, "id": 1},
    2: {"length": 2, "value": 1, "defects": {"a": 1, "b": 2, "c": 1}, "id": 2},
    3: {"length": 5, "value": 8, "defects": {"a": 2, "b": 3, "c": 2}, "id": 3},
}

In [4]:
from ortools.sat.python import cp_model

In [53]:
# 1. Construct the model
model = cp_model.CpModel()

# 2. Define the variables
# x = -1: No biscuit, x in [0, 3]: Biscuit with type x starts at position i
positions = [model.NewIntVar(-1, 3, f'biscuit_at_position_{i}') for i in range(500)]

In [54]:
# 3. Define the constraints
biscuit_type_starts_here_vars = {}

for p in range(500):
    biscuit_starts_here = model.NewBoolVar(f'biscuit_starts_here_{p}')
    model.Add(positions[p] != -1).OnlyEnforceIf(biscuit_starts_here)
    model.Add(positions[p] == -1).OnlyEnforceIf(biscuit_starts_here.Not())
    
    for biscuit_type in range(4):  # Assuming there are 4 types of biscuits indexed 0 to 3
        biscuit_type_starts_here_vars[(p, biscuit_type)] = model.NewBoolVar(f'biscuit_type_starts_here_{p}_{biscuit_type}')
        biscuit_type_starts_here = biscuit_type_starts_here_vars[(p, biscuit_type)]
        model.Add(positions[p] == biscuit_type).OnlyEnforceIf(biscuit_type_starts_here)
        model.Add(positions[p] != biscuit_type).OnlyEnforceIf(biscuit_type_starts_here.Not())
        
        # Create a constraint for the length of the biscuit starting at position p
        for i in range(biscuits[biscuit_type]["length"]):
            if p + i < 500:
                model.Add(positions[p + i] == -1).OnlyEnforceIf(biscuit_type_starts_here)


In [55]:
# 4. Define the objective function

biscuit_values = []

# for p in range(500):
#     for biscuit_type in range(4):  # Assuming there are 4 types of biscuits indexed 0 to 3
#         biscuit_starts_here = biscuit_type_starts_here_vars[(p, biscuit_type)]
#         # We already have constraints that associate biscuit_starts_here with positions[p]
        
#         # Multiply the value of the biscuit by a boolean that indicates if the biscuit of that type starts here
#         biscuit_value_contribution = model.NewIntVar(0, biscuits[biscuit_type]["value"], f'biscuit_value_contribution_{biscuit_type}_{p}')
#         model.Add(biscuit_value_contribution == biscuits[biscuit_type]["value"]).OnlyEnforceIf(biscuit_starts_here)
#         model.Add(biscuit_value_contribution == 0).OnlyEnforceIf(biscuit_starts_here.Not())
        
#         # Add the contribution to the list
#         biscuit_values.append(biscuit_value_contribution)

# The objective is to maximize the sum of biscuit values
# total_value = model.NewIntVar(0, sum(biscuit["value"] for biscuit in biscuits.values()) * 500, 'total_value')
# model.Add(total_value == sum(biscuit_values))

# Tell the model to maximize the total value
model.Add(sum(positions) == -501)
#model.Maximize(sum(biscuit_values))

In [56]:

solver = cp_model.CpSolver()
status = solver.Solve(model)

In [57]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Optimal value:", solver.ObjectiveValue())
    for p in range(500):
        print(f"Position {p}: {solver.Value(positions[p])}")
else:
    print("No solution found.")

Optimal value: 0.0
Position 0: -1
Position 1: -1
Position 2: -1
Position 3: -1
Position 4: -1
Position 5: -1
Position 6: -1
Position 7: -1
Position 8: -1
Position 9: -1
Position 10: -1
Position 11: -1
Position 12: -1
Position 13: -1
Position 14: -1
Position 15: -1
Position 16: -1
Position 17: -1
Position 18: -1
Position 19: -1
Position 20: -1
Position 21: -1
Position 22: -1
Position 23: -1
Position 24: -1
Position 25: -1
Position 26: -1
Position 27: -1
Position 28: -1
Position 29: -1
Position 30: -1
Position 31: -1
Position 32: -1
Position 33: -1
Position 34: -1
Position 35: -1
Position 36: -1
Position 37: -1
Position 38: -1
Position 39: -1
Position 40: -1
Position 41: -1
Position 42: -1
Position 43: -1
Position 44: -1
Position 45: -1
Position 46: -1
Position 47: -1
Position 48: -1
Position 49: -1
Position 50: -1
Position 51: -1
Position 52: -1
Position 53: -1
Position 54: -1
Position 55: -1
Position 56: -1
Position 57: -1
Position 58: -1
Position 59: -1
Position 60: -1
Position 61: -1